In [185]:
import numpy as np
import os
from os import listdir
import librosa as lb
import pandas as pd
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import glob
import random
import torch

In [186]:
if os.path.exists('data')==False:
    os.mkdir('data')

## Making a Experiment Folder

In [187]:
exp = 'exp-12'

In [188]:
if os.path.exists(os.path.join('data',exp))==False:
    os.mkdir(os.path.join('data',exp))
else:
    print('Folder is exists')

In [189]:
dataset = 'dataset/Merge-Data/'

## Noise Augmentation

In [190]:
noisetypes = ['noise']
musan_path = 'dataset/Musan-Data'

noisesnr   = {'noise':[0,15],'speech':[13,20],'music':[5,15]}
numnoise   = {'noise':[1,1], 'speech':[3,7],  'music':[1,1] }
noiselist  = {}
augment_files   = glob.glob(os.path.join(musan_path,'*/*/*.wav'));

for file in augment_files:
    if not file.split('/')[-3] in noiselist:
        noiselist[file.split('/')[-3]] = []
    noiselist[file.split('/')[-3]].append(file)

In [191]:
def additive_noise(noisetypes, noiselist, noisesnr, numnoise, audio, noisecat=None):
    if noisecat is None:
        noisecat = noisetypes[0]

    wav = audio.reshape(audio.shape[-1])
    clean_db = 10 * np.log10(np.mean(wav ** 2)+1e-4) 
    numnoise = numnoise[noisecat]
    noiselist = random.sample(noiselist[noisecat], random.randint(numnoise[0],numnoise[1]))

    final_noise_audio = 0
    for noise in noiselist:
        noise_wav,sr = lb.load(noise)
        noise_wav = noise_wav.reshape(noise_wav.shape[-1])
        noise_wav_len = noise_wav.shape[0]
        wav_len = wav.shape[0]
        if noise_wav_len <= wav_len:
            continue

        noise_start_slice = random.randint(0,noise_wav_len-(wav_len+1))
        noise_wav = noise_wav[noise_start_slice:noise_start_slice+wav_len]

        noise_snr = random.uniform(noisesnr[noisecat][0],noisesnr[noisecat][1])
        noise_db = 10 * np.log10(np.mean(noise_wav ** 2)+1e-4)
        final_noise_audio = np.sqrt(10 ** ((clean_db - noise_db - noise_snr) / 10)) * noise_wav

    out_audio = np.array(wav + final_noise_audio)
    out_audio = out_audio.reshape(-1)
    return out_audio

## Normalization

In [192]:
def normalization(sound,sr,scaler):
    stft = np.abs(lb.stft(sound))

    mfccs = np.mean(lb.feature.mfcc(y=sound, sr=sample_rate, n_mfcc=40),axis=1)
    mfccs_norm = scaler.fit_transform(np.reshape(mfccs,[mfccs.shape[0],1]))

    chroma = np.mean(lb.feature.chroma_stft(S=stft, sr=sample_rate),axis=1)
    chroma_norm = scaler.fit_transform(np.reshape(chroma,[chroma.shape[0],1]))

    mel = np.mean(lb.feature.melspectrogram(sound, sr=sample_rate),axis=1)
    mel_norm = scaler.fit_transform(np.reshape(mel,[mel.shape[0],1]))

    contrast = np.mean(lb.feature.spectral_contrast(S=stft, sr=sample_rate),axis=1)
    contrast_norm = scaler.fit_transform(np.reshape(contrast,[contrast.shape[0],1]))

    tonnetz = np.mean(lb.feature.tonnetz(y=lb.effects.harmonic(sound), sr=sample_rate),axis=1)
    tonnetz_norm = scaler.fit_transform(np.reshape(tonnetz,[tonnetz.shape[0],1]))

    concat = np.concatenate((mfccs,chroma,mel,contrast,tonnetz))
    concat_norm = np.concatenate((mfccs_norm,chroma_norm,mel_norm,contrast_norm,tonnetz_norm))
    
    return concat,concat_norm

In [193]:
scaler = StandardScaler()

## Extracting Train Data Feature

In [194]:
df_train = pd.read_csv('csv_files/experiment_data/train.csv')
df_train.head()

,filename,label
0,aHJPsm6esHeW5apkd8KAFdia9zj2.wav,positive
1,ptuwF1pj4ibIoVuZmt4nKb5naiZ2.wav,positive
2,EbWP0XeMpnbrTsljro0dOGKIell1_heavy.wav,positive
3,Ued1DKV2NkUnzWjJcPMxA6Q261r1_heavy.wav,positive
4,ZaeW7vuikQgf7984JVqXYOtUQMs2.wav,positive


In [ ]:
img_train = []
label_train = []

to_hot_one = {'positive':0, 'negative':1}

for index,row in df_train.iterrows():
    name = row['filename']
    label = row['label']
    filename = os.path.join(dataset,name)
    
    try:
        print(name, label)
        sound, sample_rate = lb.load(filename, sr=None)
        sound = lb.util.pad_center(sound, 30*sample_rate)
        sound_aug = additive_noise(noisetypes, noiselist, noisesnr, numnoise, audio=sound, noisecat=None)
        
        concat,concat_norm = normalization(sound,sr,scaler)
        img_train.append(concat_norm)
        label_train.append(to_hot_one[label])
        
        concat_aug,concat_aug_norm = normalization(sound_aug,sr,scaler)
        img_train.append(concat_aug_norm)
        label_train.append(to_hot_one[label])        

    except:
        print('None')

aHJPsm6esHeW5apkd8KAFdia9zj2.wav positive
ptuwF1pj4ibIoVuZmt4nKb5naiZ2.wav positive
EbWP0XeMpnbrTsljro0dOGKIell1_heavy.wav positive
Ued1DKV2NkUnzWjJcPMxA6Q261r1_heavy.wav positive
ZaeW7vuikQgf7984JVqXYOtUQMs2.wav positive
gDBsK8LcY0Om4DujIs4lFSsktWu1_heavy.wav positive
M9CdGPOCUFPWV7KnLgvStPgjLrF3_heavy.wav positive
GBwlY1I67YdQ47ceNFdpK87kEr33.wav positive
RmWwhKI8kjPi5qN9H740rfm0jHA2_heavy.wav positive
Fi2lLmV3LHR5brvXs0z1JSKDvnI2.wav positive
8JuoYlwoiUgaSSUUKQnB7F7fG5S2_heavy.wav positive
KFQ0ksPwkDcLUUjbVBYWGWr9ei62_heavy.wav positive
DRoZujDy6CWVFL0UefbHEgCcD2l2.wav positive
o8LJfYAChnd5Jv0WVa6Ni7jpHf92.wav positive
75OYElXmxdQTQne9breAIqnSo7Z2_heavy.wav positive
rB4ewmXSc9P5IPhLffnBasjYU1s2_heavy.wav positive
NVzhTN4CXebH2lQVFkBTyl6ue5h2_heavy.wav positive
BVcBPtxcv3cqEdIkhjM6YenlWtb2.wav positive
46bIeJk9J8XrD4ND2IPRMQzjJBt2_heavy.wav positive
HK3FfcI5SyXc7CtQMaQiMQ2ia1n2_heavy.wav positive
saaw7uHQJQesN9AmgKyXNFAaXB82.wav positive
etg5L3iyQ3fQAyyn4uZ9jYuoUf13_heavy.wav positiv

/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


AWMx6dOBlFPdjWa86wkLjsGZ6by2.wav positive
HK3FfcI5SyXc7CtQMaQiMQ2ia1n2.wav positive
6ZovArshO1MTe2tpxFWrHmORa2R2_heavy.wav positive
FC4RKq4W8uRrjmin8n2BXC4SrnU2.wav positive
zbIXdZwHjgb2uDtKqIgcSnr7pyD2.wav positive
x89c2X4Q5LdiRvU9XWVkU1XXLE13.wav positive
S0ZhYGoCVfXWMHzPGZ41PvHYiw12_heavy.wav positive
4bnHQCj8tnSEx37AR5HiQAH2vB32_heavy.wav positive
M43FaHpw1ScIi97eG6LBx2fBaJ92.wav positive
IimA0GnOm7ZR4nEr8k0kG4eaR7A3_heavy.wav positive
d7w3B2YcJ3TLx58ryhiASEtwaAu1.wav positive
ffrNAGEUQEfL39VAP9Eau1a3UUX2.wav positive
a96ovf71KqfbVIG8ZF5m6DaRE9Z2.wav positive
0Js6ZUZQ9NUnu568Fh7B6mZ1R8o1_heavy.wav positive


/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


saaw7uHQJQesN9AmgKyXNFAaXB82_heavy.wav positive
8Vz9jwmadkRfjbwrHwGZ2MsUtcz1_heavy.wav positive
nSVDxo9HNmPPggf6WSuAGg9XU6p1_heavy.wav positive
Y7RB11Ha8VNbBDjGoYVuIxnpYKA2_heavy.wav positive
fX1IzMBBKwXISMTgnZUFQa6o5Rh2_heavy.wav positive
31euepHD0deCxTd2nJ1wzXCk5EF3_heavy.wav positive
YHHZYBxPHsUWYITjtMRAixs3DX42.wav positive
YE3Hs5yD9BeIzT8cctZEBKGJIXN2_heavy.wav positive


...

Ny1eG1jTBwWIudUg5hGhCchHKXC2_heavy.wav positive
Y9vdHTL7XdWJVBGtktKW9gkw7yS2_heavy.wav positive
2vH4QMrMaMMm5oZYScY0b36Z8MC3_heavy.wav positive
Zmjv6hT2Q1VfJhGdGh9VkGmpHL03_heavy.wav positive
QjBZv868nydJzk0ZzwgKDHSG6Q82_heavy.wav positive
lN6Y7317RFS0apVdnffs95djXIj1.wav positive
ZIzrdWNQK1Xhn6o6jrzSVBMdphp2.wav positive
3FC5FIdqx5S5mB69VSxhduD1q0H3.wav positive
9hXEs9OejdVxG6JJGCyKQpqVvy43.wav positive
IHOunHyVaoO7AR6Z0QJgIa31BvS2.wav positive
eR2gfqeYAjdgzpwtCDXUqts9yIc2_heavy.wav positive


/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


joORHxc0iCTwgP7Uh5SM5N7rnnf2.wav positive
E4JCh6HgOVRX04QDLVotx8SgXVp1_heavy.wav positive
46bIeJk9J8XrD4ND2IPRMQzjJBt2.wav positive
uRtWGtWi3eVHJ0JhF0ycETkX0ms2_heavy.wav positive
MF5f99sONpcYl5fYsVzh32XwSmm2.wav positive
2j5I1dnLtBN8hyD149k2dZ19foE3.wav positive
Ny1eG1jTBwWIudUg5hGhCchHKXC2.wav positive
RUZ93kkyD0hJ1v7awXutr6RgF943.wav positive
0sIeyohqXMOGTqPA7RiiuyJ3AOt2.wav positive
BRVYzpZ9X1TGGLnjYEHIn3VEJ5j1.wav positive
0KSi2atlmsXNcGMfpDNzIjJSvC23_heavy.wav positive
b7mMUQm5bObj1jwGFaNyaR07pt83_heavy.wav positive
JsTOkdqknPeCRi6i63Cfu1ciuuh2.wav positive
75OYElXmxdQTQne9breAIqnSo7Z2.wav positive
ZaeW7vuikQgf7984JVqXYOtUQMs2_heavy.wav positive
tkM8UQsV2hVGrkSwelg3AyFiXoy2.wav positive
dFtGnzYqh1NVAwQUQ3wkysqAe3n1.wav positive
lQ1cBTWdazLLa7xWAiSCOmZ709s1.wav positive
kBFDtvAVY9QYbi7YHYgd7tNpsWx1.wav positive
xce3uTKiuNSSnnAWLJMorxTjHzI2_heavy.wav positive
UZKdFMia0JWykTWi6T2PoXmS3nt2.wav positive
yL9QW3T2VueXfL3yvsmXsM2hL4v1_heavy.wav positive
08tlSbvXpAggZavIGW994adhh1m1.wav p

/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/home/viblab/.local/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


CkBv7ZLYRIcugw93mPtPkbWxC412.wav positive
zM5GsorprLYElGQ3uJwnU0g0Fwr1_heavy.wav positive
lpohIzcDNEg3DR38ku43WHiCkC42.wav positive
lN6Y7317RFS0apVdnffs95djXIj1_heavy.wav positive
dvRr97gwv7Y34YqaccErnWcYRY53_heavy.wav positive
h5e7Ff7hWON2RPULdDCxad7xpGv2_heavy.wav positive
imaxvQ1daYhtvhpulzgQy4t6VMl2_heavy.wav positive
qJvoAC408vP9fwBWQUAHHxKI39k1.wav positive
333NjqA1TfZJuICEdXSkPhVz0LA3.wav positive
XwPxN3JASwgk13QZUviNawM7k0v1.wav positive
Qcliznd3z1VdWmJOZh9nvlstTYv1.wav positive
wEMYsNrzUvXhlQHYnFAF6U5a0Py1_heavy.wav positive
jyxqyMzm2IXXaETlfHjV1K6ju492.wav positive
2tjg2NfgJPaYdgTZRDUBhWm8F133_heavy.wav positive
QjBZv868nydJzk0ZzwgKDHSG6Q82.wav positive
XFlqN1sDBKNXyuORhVagkJnr5V33.wav positive
cLcriOknOebRZSFfeQlcUNYzbZL2.wav positive
khBKxUEe1rQxPqpo4pGylAZjsEt1_heavy.wav positive
Cd7fkliPM2ZUM1skytvoU2vD8jK2_heavy.wav positive
YHHZYBxPHsUWYITjtMRAixs3DX42_heavy.wav positive
euld60aqc2OtnNoKYx9P4WDzVM92_heavy.wav positive
05acPS4aRGfvuOfku11Za8zve8i2.wav positive
T190nKNqKo

In [ ]:
np.save(os.path.join('data',exp,'img_train.npy'),img_train)
np.save(os.path.join('data',exp,'label_train.npy'),label_train)

In [ ]:
img_train = np.load(os.path.join('data',exp,'img_train.npy'))
label_train = np.load(os.path.join('data',exp,'label_train.npy'))

In [ ]:
label_train.shape

## Extracting Devel Data Feature

In [ ]:
df_devel = pd.read_csv('csv_files/experiment_data/devel.csv')
df_devel.head()

In [ ]:
img_devel = []
label_devel = []

to_hot_one = {'positive':0, 'negative':1}

for index,row in df_devel.iterrows():
    name = row['filename']
    label = row['label']
    filename = os.path.join(dataset,name)
    
    try:
        print(name, label)
        sound, sample_rate = lb.load(filename, sr=None)
        sound = lb.util.pad_center(sound, 30*sample_rate)
        sound_aug = additive_noise(noisetypes, noiselist, noisesnr, numnoise, audio=sound, noisecat=None)
        
        concat,concat_norm = normalization(sound,sr,scaler)
        img_devel.append(concat_norm)
        label_devel.append(to_hot_one[label])
        
        concat_aug,concat_aug_norm = normalization(sound_aug,sr,scaler)
        img_devel.append(concat_aug_norm)
        label_devel.append(to_hot_one[label])  
    except:
        print('None')

In [ ]:
np.save(os.path.join('data',exp,'img_devel.npy'),img_devel)
np.save(os.path.join('data',exp,'label_devel.npy'),label_devel)

In [ ]:
img_devel = np.load(os.path.join('data',exp,'img_devel.npy'))
label_devel = np.load(os.path.join('data',exp,'label_devel.npy'))

## Extracting Test Data Feature 

In [ ]:
df_test = pd.read_csv('csv_files/experiment_data/test.csv')
df_test.head()

In [ ]:
img_test = []
label_test = []

to_hot_one = {'positive':0, 'negative':1}

for index,row in df_test.iterrows():
    name = row['filename']
    label = row['label']
    filename = os.path.join(dataset,name)
    
    try:
        print(name, label)
        sound, sample_rate = lb.load(filename, sr=None)
        sound = lb.util.pad_center(sound, 30*sample_rate)
        
        concat,concat_norm = normalization(sound,sr,scaler)
        img_test.append(concat_norm)
        label_test.append(to_hot_one[label])
    except:
        print('None')

In [ ]:
np.save(os.path.join('data',exp,'img_test.npy'),img_test)
np.save(os.path.join('data',exp,'label_test.npy'),label_test)

In [ ]:
img_test = np.load(os.path.join('data',exp,'img_test.npy'))
label_test = np.load(os.path.join('data',exp,'label_test.npy'))

## Load Data

In [ ]:
img_train = np.load(os.path.join('data',exp,'img_train.npy'))
label_train = np.load(os.path.join('data',exp,'label_train.npy'))

img_devel = np.load(os.path.join('data',exp,'img_devel.npy'))
label_devel = np.load(os.path.join('data',exp,'label_devel.npy'))

img_test = np.load(os.path.join('data',exp,'img_test.npy'))
label_test = np.load(os.path.join('data',exp,'label_test.npy'))

In [ ]:
img_train = np.array(img_train)
label_train = np.array(label_train)

img_devel = np.array(img_devel)
label_devel = np.array(label_devel)

img_test = np.array(img_test)
label_test = np.array(label_test)

In [ ]:
print(img_train.shape)
print(img_devel.shape)
print(img_test.shape)

In [ ]:
if os.path.exists(os.path.join('data',exp,'array'))==False:
    os.mkdir(os.path.join('data',exp,'array'))

## Save data array\

In [ ]:
np.save(os.path.join('data',exp,'array','img_train.npy'),img_train)
np.save(os.path.join('data',exp,'array','label_train.npy'),label_train)

np.save(os.path.join('data',exp,'array','img_devel.npy'),img_devel)
np.save(os.path.join('data',exp,'array','label_devel.npy'),label_devel)

np.save(os.path.join('data',exp,'array','img_test.npy'),img_test)
np.save(os.path.join('data',exp,'array','label_test.npy'),label_test)

In [ ]:
img_train = np.load(os.path.join('data',exp,'array','img_train.npy'))
label_train = np.load(os.path.join('data',exp,'array','label_train.npy'))

img_val = np.load(os.path.join('data',exp,'array','img_devel.npy'))
label_val = np.load(os.path.join('data',exp,'array','label_devel.npy'))

img_test = np.load(os.path.join('data',exp,'array','img_test.npy'))
label_test = np.load(os.path.join('data',exp,'array','label_test.npy'))

## Augmented using SMOTE

In [ ]:
oversample = SMOTE()
img_train_smote, label_train_smote = oversample.fit_resample(np.reshape(img_train,[img_train.shape[0],img_train.shape[1]]), label_train)

In [ ]:
oversample = SMOTE()
img_val_smote, label_val_smote = oversample.fit_resample(np.reshape(img_val,[img_val.shape[0],img_val.shape[1]]), label_val)

In [ ]:
np.save(os.path.join('data',exp,'array','img_train_smote.npy'),img_train_smote)
np.save(os.path.join('data',exp,'array','label_train_smote.npy'),label_train_smote)

np.save(os.path.join('data',exp,'array','img_devel_smote.npy'),img_val_smote)
np.save(os.path.join('data',exp,'array','label_devel_smote.npy'),label_val_smote)